## Setup

### Imports

In [1]:
# imports
import os
from os.path import isdir, join
import shutil
import re
import glob
import pandas as pd
from tqdm import tqdm
from pypdf import PdfReader
from src.utils import *
from src.process_reports import *

### Settings

In [39]:
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_rows', 100)

### Restructure tcga data folders by case

In [5]:
%%script false --no-raise-error
# define the base directory
base_dir = '/mnt/disks/ext/data/gdc/tcga/brca/'
# Define the pattern for the case id
pattern = r"TCGA-\w{2}-\w{4}"
# Iterate over all directories in the base directory
for dirpath, dirnames, filenames in os.walk(base_dir):
    for filename in filenames:
        # Find the case id in the filename
        match = re.search(pattern, filename)
        if match:
            case_id = match.group()
            # Create a new directory for this case id, if it doesn't exist
            new_dir = os.path.join(base_dir, case_id)
            os.makedirs(new_dir, exist_ok=True)
            # Move the file to the new directory
            shutil.move(os.path.join(dirpath, filename), os.path.join(new_dir, filename))

# move other folders/files, except for the case folders, to misc folder
# Define the pattern for the case id and the 8-digit alphanumeric
case_pattern = r"TCGA-\w{2}-\w{4}"
misc_pattern = r"^[a-z0-9]{8}-"

# Create the 'misc' directory if it doesn't exist
misc_dir = os.path.join(base_dir, 'misc')
os.makedirs(misc_dir, exist_ok=True)

# Create a list to store directories to be moved
dirs_to_move = []

# Generate a list of all directories in the base directory
all_dirs = [x[0] for x in os.walk(base_dir) if not x[0].startswith(misc_dir)]

# Iterate over all directories in the list
for dirpath in all_dirs:
    # If the directory is empty, remove it
    if dirpath != base_dir and not any(os.scandir(dirpath)):
        os.rmdir(dirpath)
    else:
        # If the directory name starts with an 8-digit alphanumeric followed by a hyphen
        # and it's not a case directory, add it to the list of directories to be moved
        dirname = os.path.basename(dirpath)
        if re.match(misc_pattern, dirname) and not re.match(case_pattern, dirname):
            dirs_to_move.append(dirpath)

# Move the directories in the list to the 'misc' directory
for dirpath in dirs_to_move:
    if os.path.exists(dirpath):  # Check if the directory still exists
        dest_dir = os.path.join(misc_dir, os.path.basename(dirpath))
        if os.path.exists(dest_dir):
            shutil.rmtree(dest_dir)
        shutil.move(dirpath, dest_dir)
# count # cases in base dir
case_count = 0

# Iterate over all directories in the base directory
for dirpath, dirnames, _ in os.walk(base_dir):
    # If the directory name matches the case id pattern, increment the counter
    dirname = os.path.basename(dirpath)
    if re.match(case_pattern, dirname):
        case_count += 1

print(f"total number of case directories: {case_count}")

### Convert pdf to text


In [6]:
%%script false --no-raise-error
# Call the function
extract_text_from_pdf('/mnt/disks/ext/data/gdc/tcga/brca/')

### Create manifest file for feature extraction

In [3]:
# %%script false --no-raise-error
# set data dir
data_dir = '/mnt/disks/ext/data/gdc/tcga/brca/'
 
stils_tcga_elg_path = 'data/stils/stils_tcga_ellogon.tsv'
wsis_stils_elg = pd.read_csv(stils_tcga_elg_path, sep='\t')['wsi_id'].tolist()
print(f'# wsis w/ sTIL annotations: {len(wsis_stils_elg)}')

# get list of files in manifest that are in data dir
# annotated_wsis = [f for f in wsis_stils_elg if f in wsis_all]

# reference_file = '/home/neil/multimodal/data/stils_tcga_brca_annotated.txt'
wsi_stils_feats_manifest_path = 'data/stils/wsi_stils_feats_manifest.txt'
wsi_feats_manifest_path = 'data/wsi_feats_manifest.txt'

# Initialize the list for the manifest
wsi_stils_annot_paths, wsi_paths = [], []

# Loop through all case folders in data_dir
# case_ids_annot = []
for root, dirs, files in os.walk(data_dir):
    # Filter for .svs files
    all_wsi_paths = [file for file in files if file.endswith('.svs')]
    diag_wsi_paths = [file for file in files if file.endswith('.svs') and 'DX1' in file]
    if all_wsi_paths:
        # Check if any of the .svs files are in the reference file
        for wsi_path in all_wsi_paths:
            if wsi_path.replace('.svs', '') in wsis_stils_elg:
                wsi_stils_annot_paths.append(os.path.join(os.path.basename(root), wsi_path))
                # case_ids_annot.append(os.path.basename(root))
                # break
    if diag_wsi_paths:
        wsi_paths.extend([os.path.join(os.path.basename(root), wsi_path) for wsi_path in diag_wsi_paths])
# print(f'# annotated cases: {len(set(case_ids_annot))}')

# Save the list of files to the manifest path
print(f'# annotated wsis in manifest: {len(wsi_stils_annot_paths)}')
with open(wsi_stils_feats_manifest_path, 'w') as f:
    for item in wsi_stils_annot_paths:
        f.write("%s\n" % item)
with open(wsi_feats_manifest_path, 'w') as f:
    for item in wsi_paths:
        f.write("%s\n" % item)

# wsis w/ sTIL annotations: 700
# annotated wsis in manifest: 700


### Copy feature embs to/from case folder

In [33]:
%%script false --no-raise-error
# Copy feature embs from WSIs & reports to case folders
# Directory where the feature embeddings are stored
wsi_feats_dir = "data/wsi_feats"
report_feats_dir = "data/report_feats"

# Directory where the case folders are located
dst_dir = "/mnt/disks/ext/data/gdc/tcga/brca"

# Loop over all img feat files in the source directory
for src_file in glob.glob(os.path.join(wsi_feats_dir, "TCGA-*.pt")):
    # Extract the base name of the file
    base_name = os.path.basename(src_file)

    # Construct the destination directory path
    dst_file = os.path.join(dst_dir, base_name[:12], base_name)

    # Copy the file if it doesn't already exist
    if not os.path.exists(dst_file):
        shutil.copy(src_file, dst_file)
    
# Loop over all text feat files in the source directory
for src_file in glob.glob(os.path.join(report_feats_dir, "TCGA-*.pt")):
    # Extract the base name of the file
    base_name = os.path.basename(src_file)

    # Construct the destination directory path
    dst_file = os.path.join(dst_dir, base_name[:12], base_name)

    # Copy the file if it doesn't already exist
    if not os.path.exists(dst_file):
        shutil.copy(src_file, dst_file)

In [24]:
%%script false --no-raise-error
# Copy feature embs from WSIs & reports from case folders to separate folders
# Directory where the case folders are located
src_dir = "/mnt/disks/ext/data/gdc/tcga/brca"

# Directory where the feature embeddings will be stored
wsi_feats_dir = "data/wsi_feats"
report_feats_dir = "data/report_feats"

# Create the directories if they don't exist
os.makedirs(wsi_feats_dir, exist_ok=True)
os.makedirs(report_feats_dir, exist_ok=True)

# Loop over all case folders in the source directory
for case_folder in glob.glob(os.path.join(src_dir, "TCGA-*")):
    # Loop over all img feat files in the case folder
    for src_file in glob.glob(os.path.join(case_folder, "*.wsi.pt")):
        # Extract the base name of the file
        base_name = os.path.basename(src_file)

        # Construct the destination file path
        dst_file = os.path.join(wsi_feats_dir, base_name)

        # Copy the file
        shutil.copy(src_file, dst_file)

    # Loop over all text feat files in the case folder
    for src_file in glob.glob(os.path.join(case_folder, "*.report.pt")):
        # Extract the base name of the file
        base_name = os.path.basename(src_file)

        # Construct the destination file path
        dst_file = os.path.join(report_feats_dir, base_name)

        # Copy the file
        shutil.copy(src_file, dst_file) 

### Create .csv file for loading dataset

In [ ]:
%%script false --no-raise-error
# for sTILs dataset
# Load the annotations
wsi_stils_annot = pd.read_csv('data/stils/stils_tcga_ellogon.tsv', sep='\t')

# Set data dirs
wsi_feats_dir = 'data/wsi_feats'
report_feats_dir = 'data/report_feats'

# Initialize the dataset
data_stils = []

# Define the pattern for the case id
case_pattern = r"TCGA-\w{2}-\w{4}"

# Walk through the wsi_feats_dir
for root, dirs, files in os.walk(wsi_feats_dir):
    for wsi_feat_file in files:
        # Check if the file is a feature file
        if wsi_feat_file.endswith('.wsi.pt'):
            # Extract the case id and slide id
            case_id = wsi_feat_file.split('.wsi.pt')[0][:12]
            slide_id = wsi_feat_file.split('.wsi.pt')[0]
            
            # print(f'case_id: {case_id}, slide_id: {slide_id}')
            
            # Find the matching row in the annotations
            annot = wsi_stils_annot[wsi_stils_annot['wsi_id'] == slide_id]
            split, stil_score = annot['split'].values[0], annot['stil_score'].values[0] if not annot.empty else (None, None)
            
            # Find the report file
            report_feat_file = next((f for f in os.listdir(report_feats_dir) if f.startswith(case_id) and f.endswith('.report.pt')), None)
            if report_feat_file is not None:
                report_feat_path = os.path.join(report_feats_dir, report_feat_file)
                wsi_feat_path = os.path.join(wsi_feats_dir, wsi_feat_file)
                # Add the data to the dataset
                data_stils.append([case_id, wsi_feat_path, report_feat_path, split, stil_score])


# Convert the dataset to a DataFrame and save it to a CSV file
df_data_stils = pd.DataFrame(data_stils, columns=['case_id', 'wsi_feat_path', 'report_feat_path', 'split', 'stil_score'])

# drop rows w/ no sTIL score
df_data_stils.dropna(subset=['stil_score'], inplace=True)

# df['set'] = df['set'].replace({'Training': 'train', 'Test': 'test', 'Validation': 'val'})

# bucketize sTIL scores
df_data_stils['stil_lvl'] = df_data_stils['stil_score'].apply(lambda x: int(x // 0.1))

print(f'# samples: {len(df_data_stils)}')
df_data_stils.head()

# save dataset to csv
df_data_stils.to_csv('data/stils/data_stils.csv', index=False)

# dataset.head(20)

In [52]:
%%script false --no-raise-error
# for subtype & grade dataset
# Load the annotations
reports_annot = pd.read_csv('data/data_subtype_grade_annot_clean.csv', dtype='str', keep_default_na=False)

# Set data dirs
wsi_feats_dir = 'data/wsi_feats'
report_feats_dir = 'data/report_feats'

# Initialize the dataset
data_subtype_grade_annot = []

# Define the pattern for the case id
case_pattern = r"TCGA-\w{2}-\w{4}"

for case_id in reports_annot['case_id'].unique():
    # read subtype & grade
    split, region, localization, grade = reports_annot[reports_annot['case_id'] == case_id][['split', 'region', 'localization', 'grade']].values[0]
    
    # find wsi feat file
    wsi_feat_file = next((f for f in os.listdir(wsi_feats_dir) if f.startswith(case_id) and f.endswith('.wsi.pt')), None)
    if wsi_feat_file is not None:
        wsi_feat_path = os.path.join(wsi_feats_dir, wsi_feat_file)
    
    # Find the report file
    report_feat_file = next((f for f in os.listdir(report_feats_dir) if f.startswith(case_id) and f.endswith('.report.pt')), None)
    if report_feat_file is not None:
        report_feat_path = os.path.join(report_feats_dir, report_feat_file)

    # Add the data to the dataset
    data_subtype_grade_annot.append([case_id, wsi_feat_path, report_feat_path, split, region, localization, grade])
    
# Convert the dataset to a DataFrame and save it to a CSV file
df_subtype_grade = pd.DataFrame(data_subtype_grade_annot, columns=['case_id', 'wsi_feat_path', 'report_feat_path', 'split', 'region', 'localization', 'grade'], dtype=object)

print(f'# samples: {len(df_subtype_grade)}')
df_subtype_grade.head()

# save dataset file
df_subtype_grade.to_csv('data/data_subtype_grade.csv', index=False)

# samples: 1069


### Rename report files

In [ ]:
reports_dir = 'data/reports_processed'

# Loop through each file in the directory
for filename in os.listdir(reports_dir):
    # Check if the filename matches the pattern 'TCGA-XX-XXXX.*.txt'
    if filename.startswith('TCGA-') and filename.endswith('.txt'):
        # Extract the case_id (first 12 characters)
        case_id = filename[:12]
        new_filename = f"{case_id}.txt"
        
        # Construct the full paths for the source and destination
        src_path = os.path.join(reports_dir, filename)
        dest_path = os.path.join(reports_dir, new_filename)
        
        # Check if the destination filename already exists
        if os.path.exists(dest_path):
            print(f"destination filename {dest_path} already exists!")
        else:
            # Rename the file
            os.rename(src_path, dest_path)
        

### Extract relevant info (summarize) from reports

In [5]:
# Define your directories
reports_dir = 'data/reports'
summary_dir = 'data/reports_distilled'
res_dir = 'data/reports_residual'

# Extract and save the reports
distill_reports(reports_dir, summary_dir, res_dir)

### Preprocess subtype & grade data

In [ ]:
df_subtype_grade = pd.read_csv('data/data_subtype_grade_annot.csv')
print(df_subtype_grade.head(10))
# get value counts for each region, localization and grade
print(f'Value counts: {[df_subtype_grade[key].value_counts() for key in ["region", "localization", "grade"]]}')

In [ ]:
# clean data using openai api
report_data_file = 'data/data_subtype_grade_annot.csv'
with open(report_data_file, 'r') as f:
    report_text = f.read()

# extract header
report_header, report_body = report_text.split('\n', 1)

# construct prompt for lm
context = '''This csv file contains annotations for cancer subtypes and grades. The file is unstructured. I want to standardize it like so:
- for each of the 3 categories/cols (region, localization, grade), the corresponding labels should all be converted to lowercase)
- for 'region', the valid labels are 'ductal', 'lobular', 'mixed' and 'NA'. all entries like 'intraductal' 'ductal (intraductal)' , 'Infiltrating ductal' etc. should all be converted to just 'ductal'. but if both 'lobular' and 'ductal' occur in the label, then convert it to 'mixed'. any other labels should be converted to 'NA'
- for 'localization', the valid labels are 'in situ', 'invasive' , 'metastatic' and 'NA'. all entries like 'infiltrating', 'infiltrating/invasive ', 'infiltrating (invasive)' should be converted to just 'invasive'. any labels which are not one of 'invasive', 'in situ' or 'metastatic' should be converted to 'NA'.
- for 'grade', the valid labels are '1', '2', '3' and 'NA'. so all labels like 'I', 'well differentiated', '1, (well differentiated)', 'grade 1', 'grade I' etc. should be converted to just '1', and the same for 2 (moderately differentiated) and 3 (poorly differentiated). If there are multiple grades (1/2) or a range (2-3), convert to the higher one. finally, all of 'insufficient information' etc. should be converted to just 'NA'. 
Based on the above instructions, please convert the labels in the csv file below to the standardized format and return the cleaned csv file as a string.\n'''

max_prompt_len = 5000
max_context_len = max_prompt_len - len(context)

# split prompt into chunks of max_context_len
prompt_chunks = [context + report_header + '\n' + report_body[i:i+max_context_len] for i in range(0, len(report_body), max_context_len)]
# prompt = f'''\n {report_text}'''
    
# init lm
lm_name = "gpt-3.5-turbo"
gen_args = {} # default args for generation

# call openai
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables.")

API_URL = "https://api.openai.com/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}

outputs = []

for prompt_chunk in tqdm(prompt_chunks):
    data = {
        "model": lm_name,
        "messages": [{"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt_chunk}],
        **gen_args  # Add any additional arguments
    }

    response = requests.post(API_URL, headers=headers, json=data)
    # print(response.content)
    response.raise_for_status()
    out = response.json()["choices"][0]["message"]["content"]
    outputs.append(out)
    
# join outputs (and remove header in all but first chunk)
output_text = outputs[0] + '\n' + '\n'.join([out.split('\n', 1)[1] for out in outputs[1:]])
print(f'cleaned annotations: {output_text}')

# write cleaned data to file
with open('data/data_subtype_grade_annot_clean.csv', 'w') as f:
    f.write(output_text)

In [48]:
df_subtype_grade = pd.read_csv('data/data_subtype_grade_annot_clean.csv', keep_default_na=False)
print(df_subtype_grade.head(10))

# validate data
valid_region_labels = ['ductal', 'lobular', 'mixed', 'NA']
valid_localization_labels = ['in situ', 'invasive', 'metastatic', 'NA']
valid_grade_labels = ['1', '2', '3', 'NA']

# convert all 'other/NA' to 'NA'
# df_subtype_grade.replace('other/NA', 'NA', inplace=True)

# print invalid labels & rows
print(f'Invalid labels: {df_subtype_grade[~df_subtype_grade.region.isin(valid_region_labels) | ~df_subtype_grade.localization.isin(valid_localization_labels) | ~df_subtype_grade.grade.isin(valid_grade_labels)]}')

# create a 'split' column with a random train/val/test split (80/10/10)
from sklearn.model_selection import train_test_split

# Split the data into train (80%) and temp (20%)
train_df, temp_df = train_test_split(df_subtype_grade, test_size=0.2, random_state=42)
# Split the temp data into validation (50%) and test (50%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
# Add a 'split' column to each dataframe
train_df['split'], val_df['split'], test_df['split'] = 'train', 'val', 'test'
# Concatenate the dataframes back together
df_subtype_grade = pd.concat([train_df, val_df, test_df])

# print(df_subtype_grade.head(10))
# print(f'Value counts: {[df_subtype_grade[key].value_counts() for key in ["split"]]}')

# save cleaned data
df_subtype_grade.to_csv('data/data_subtype_grade_annot_clean.csv', index=False)

        case_id   region localization grade
0  TCGA-3C-AAAU  lobular     invasive     3
1  TCGA-3C-AALI   ductal     invasive     2
2  TCGA-3C-AALJ   ductal     invasive     3
3  TCGA-3C-AALK   ductal     invasive    NA
4  TCGA-4H-AAAK  lobular     invasive     2
5  TCGA-5L-AAT0  lobular     invasive     1
6  TCGA-5L-AAT1  lobular     invasive     1
7  TCGA-5T-A9QA    mixed     invasive     3
8  TCGA-A1-A0SB       NA     invasive     1
9  TCGA-A1-A0SD   ductal     invasive     2
Invalid labels: Empty DataFrame
Columns: [case_id, region, localization, grade]
Index: []
